In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn import set_config
set_config(display = 'diagram')

In [2]:
df = pd.read_csv('D:/Python/Datasets/Titanic Dataset/titanic_full.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=0, inplace = True)

In [5]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [6]:
df.shape

(891, 8)

In [7]:
# Step 1: train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'],
                                                   test_size = 0.2, random_state=42)

In [8]:
# Step 2: Imputing Transformer 
# Column 2 (Age) and 6 (Embarked)

trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6])
], remainder = 'passthrough')

In [9]:
# Step 3: One Hot Encoding on Categorical values
# Column 1 (Sex) and 6 (Embarked)

trf2 = ColumnTransformer([
    ('ohe_sex', OneHotEncoder(sparse=False, handle_unknown='ignore'), [1, 6])
], remainder= 'passthrough')

In [10]:
# Step 4: Scaling the values
# On all Columns (After One Hot Encoding)

trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))
])

In [11]:
# Step 5: Feature Selection

trf4 = SelectKBest(score_func=chi2, k= 8)

In [12]:
# Step 6: Creating the object of the model - Decision Tree Classifier

trf5 = DecisionTreeClassifier()

Creating the Pipeline

In [13]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)
])

In [14]:
# Training the Model

pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001C2F1030160>)),
                ('trf5', DecisionTreeClassifier())])

In [15]:
# mean of the age column by simple imputer
pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([29.49884615])

In [16]:
# Predict

y_pred = pipe.predict(X_test)

In [17]:
y_pred

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0], dtype=int64)

In [18]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.6256983240223464

In [19]:
# Exporting the pipeline

import pickle
pickle.dump(pipe, open('D:/Python/Datasets/Titanic Dataset/pipe.pkl', 'wb'))